In [1]:
import numpy as np
import pandas as pd

In [2]:
N = 11

# Матрицы

In [3]:
def determinant(matrix, mul):
    width = len(matrix)
    if width == 1:
        return mul * matrix[0][0]
    else:
        sign = -1
        s = 0
        for i in range(width):
            m = []
            for j in range(1, width):
                buff = []
                for k in range(width):
                    if k != i:
                        buff.append(matrix[j][k])
                m.append(buff)
            sign *= -1
            s += mul * determinant(m, sign * matrix[0][i])
    return s

In [4]:
class Array:
    
    def __init__(self, a):
        self.array = np.array(a, dtype = 'float')
        self.shape = self.shape_f()

    def shape_f(self):
        return self.array.shape
    
    def T(self):
        res = np.zeros((self.shape[1], self.shape[0]))
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                res[j][i] = self.array[i][j]
        return Array(res)
        

    def plus(self, other):
        res = np.array(self.array)
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                res[i][j] += other.array[i][j]
        return Array(res)

    def pointwise_multiply(self, other):
        res = np.array(self.array)
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                res[i][j] *= other.array[i][j]
        return Array(res)
    

    def matrix_multiply(self, other):
        res = np.zeros((self.shape[0], other.shape[1]))
        for i in range(self.shape[0]):
            for j in range(other.shape[1]):
                for k in range(len(self.array[i])):
                    res[i][j] += self.array[i][k] * other.array[k][j]
        return Array(res)
    def det(self):
#         if self.shape[0] == self.shape[1]:
#             a = np.array(self.array)
#             res = a[0][0] * a[1][1] * a[2][2] + a[0][1] * a[1][2] * a[2][0] + a[1][0] * a[2][1] * a[0][2] - \
#                   (a[0][2] * a[1][1] * a[2][0] + a[0][1] * a[1][0] * a[2][2] + a[1][2] * a[2][1] * a[0][0])
#             return res
#         else: return 0
        return determinant(self.array, 1)
        
    
    # Operators
    def __add__(self, other):
        return self.plus(other)
    
    def __sub__(self, other):
        return self.plus(Array(-(other.array)))
    
    def __mul__(self, other):
        return self.pointwise_multiply(other)
    
    def __matmul__(self, other):
        return self.matrix_multiply(other)
    
    def __getitem__(self, ij):
        (i, j) = ij
        return self.array[i][j]

# Нормы

In [5]:
def sqrt(x, eps = 10**(-9), delta=1):
    p = (1+x)/2 
    p1 = (p+x/p)/2
    while abs(p-p1) > eps/delta:
        p = p1
        p1 = (p+x/p)/2
    return p1

def v_norm_inf(v):
    return max(v.array)
def v_norm_1(v):
    return sum(abs(v.array))
def v_norm_2(v):
    return np.sqrt((v.T() @ v).array[0][0])

def m_norm_inf(A):
    return max(sum(abs(A.array)))
def m_norm_1(A):
    return m_norm_inf(A.T())
def m_norm_2(A):
    M = A.T() @ A
    return sqrt(max(np.linalg.eigh(M.array)[0]))
def m_norm_f(A):
    return sum(sum((A*A).array))
    

m_norms = [m_norm_1, m_norm_2, m_norm_inf, m_norm_f]
v_norms = [v_norm_1, v_norm_2, v_norm_inf]
m_n_norm = 1
v_n_norm = 1

# Метод простой итерации

In [6]:
def simple_iteration(A, b, eps):
    if not A.det() :
        print("Матрица А особая")
        return 0
    b = b.T()
    n = A.shape[0]
    mu = 1/m_norms[m_n_norm](A)
    Mu = Array(np.ones(A.shape) * mu)
    B = Array(np.eye(n)) - Mu*A
    if m_norms[m_n_norm](B)>1:
        b = A.T() @ b
        A = A.T() @ A
        mu = 1/m_norms[m_n_norm](A) 
        Mu = Array(np.ones(A.shape) * mu)
        B = Array(np.eye(n)) - Mu*A
        if m_norms[m_n_norm](B)>1:
            print('Norm B > 1')
            return 0
        
    Mu = Array(np.ones(b.shape) * mu)
    c = Mu * b
    x_old = c
    x = B @ x_old + c
    
    B_norm = m_norms[m_n_norm](B)/(1-m_norms[m_n_norm](B))
    iterations = 1
    while B_norm * v_norms[v_n_norm](x-x_old) > eps:
        x_old = x
        x = B @ x_old + c
        iterations +=1
    return x, iterations

# Метод Зейделя

In [7]:
def seidel(A, b, eps):
    if not A.det() :
        print("Матрица А особая")
        return 0
    n = A.shape[0]
    b = b.T()
    
    M = [abs(A[i, i]) > (sum(abs(A.array[i]))-abs(A[i, i])) for i in range(n)]
    if sum(M) != n:
        b = A.T() @ b
        A = A.T() @ A
    C = Array(A.array)
    C = Array([C.array[i]*(-1/A[i, i]) for i in range(n)]) + Array(np.eye(n))
    d = Array([[b[i, 0]/A[i, i] for i in range(n)]]).T()
    x_old = Array(d.array)
    x = Array(x_old.array)
    for i in range(n):
        x_old = Array(x.array)
        x.array[i] = (Array([C.array[i]]) @ x_old)[0, 0] + d[i, 0]
    iterations = 1
    while v_norms[v_n_norm](A @ x - b) > eps:
        for i in range(n):
            x_old = Array(x.array)
            x.array[i] = (Array([C.array[i]]) @ x_old)[0, 0] + d[i, 0]
        iterations += 1
    return x, iterations

# Метод Гаусса

In [8]:
def gauss(A, b):
    if not A.det() :
        print("Матрица А особая")
        return 0
    A = Array(A.array)
    n = A.shape[0]
    b = b.T()
    for i in range(n):
        T = A.T()
        m = np.argmax(abs(T.array[i][i:]))
        A.array[i], A.array[i+m] = np.array(A.array[i+m]), np.array(A.array[i])
        b.array[i], b.array[i+m] = np.array(b.array[i+m]), np.array(b.array[i])
        a = A[i, i]
        A.array[i] = A.array[i]*(1/a)
        b.array[i] = b.array[i]*(1/a)
        for j in range(i+1, n):
            b.array[j] = np.array(b.array[j] - A[j, i] * b.array[i])
            A.array[j] = np.array(A.array[j] - A[j, i] * A.array[i])
    x = Array(np.zeros(b.shape))
    x.array[n-1] = b.array[n-1]
    for i in range(n-1):
        tmp = Array(-x.array)
        x.array[n-2-i] = (Array([A.array[n-2-i]]) @ tmp)[0,0] + b.array[n-2-i]
    return x

# Метод Хаусхолдера

In [9]:
def householder(A, b):
    if not A.det() :
        print("Матрица А особая")
        return 0
    A = Array(A.array)
    n = A.shape[0]
    b = b.T()
    
    Q = Array(np.eye(n))
    Ri = Array(A.array)
    R = [Ri]
    
    for i in range(n-1):
        Q_old = Array(np.eye(n))
        R_old = Array(Ri.array[i:, i:])
        zi = Array(np.zeros((n-i, 1)))
        zi.array[0][0] = 1
        yi = Array([R_old.array[:, 0]]).T()
        alpha = v_norm_2(yi)
        alp = Array([np.ones(zi.shape) * alpha]).T()
        ro = v_norm_2(yi-alp*zi)
        wi = yi - alp*zi
        wi = Array([np.ones(wi.shape)* (1/ro)]).T() * wi
        Qi = Array(np.eye(n-i)) - wi @ wi.T() - wi @ wi.T()
        Q_old.array[i:, i:] = Qi.array
        Ri.array[i:, i:] = (Qi @ R_old).array
        Q = Q_old @ Q
        R.append(R[i])
    y = Q @ b
    x = gauss(R[-1], y.T())
    return x

# Выводы

In [10]:
# Example
A_t = Array([[3, 1, 1], [1, 5, 1], [1, 1, 7]])
b_t = Array([[5, 7, 9]])

In [11]:
# Example 2
A_t2 = Array([[0, 1, 1], [1, 5, 1], [1, 1, 7]])
b_t2 = Array([[2, 20, 12]])

In [12]:
# Test 0
A0 = Array([[0, 2, 3], [1, 2, 4], [4, 5, 6]])
b0 = Array([[13, 17, 32]])

In [13]:
# Test 1
A1 = Array([[N+2, 1, 1], [1, N+4, 1], [1, 1, N+6]])
b1 = Array([[N+4, N+6, N+8]])

In [14]:
# Test 2
A2 = Array([[-(N+2), 1, 1], [1, -(N+4), 1], [1, 1, -(N+6)]])
b2 = Array([[-(N+4), -(N+6), -(N+8)]])

In [15]:
# Test 3
A3 = Array([[-(N+2), N+3, N+4], [N+5, -(N+4), N+1], [N+4, N+5, -(N+6)]])
b3 = Array([[N+4, N+6, N+8]])

In [16]:
# Test 4
A4 = Array([[N+2, N+1, N+1], [N+1, N+4, N+1], [N+1, N+1, N+1]])
b4 = Array([[N+4, N+6, N+8]])

In [17]:
A = [A0, A1, A2, A3, A4]
b = [b0, b1, b2, b3, b4]

In [18]:
nt = 5
tests = list(range(nt))

x0 = Array([[1, 2, 3]])
x1 = Array([[1, 1, 1]])
x2 = Array([[1.353, 1.308, 1.274]])
x3 = Array([[1.3, 1.119, 1.082]])
x4 = Array([[-4, -0.667, 6.25]])
xs = [x0, x1, x2, x3, x4]

epsilons = [10**(-i) for i in range(2, 5)]

## Метод простой итерации

In [33]:
x_my = []
k = []
x_true = []
errors = []
epss = []
testss = []
for i in tests:
    for epsilon in epsilons:
        x_my.append(simple_iteration(A[i], b[i], epsilon)[0])
        k.append(simple_iteration(A[i], b[i], epsilon)[1])
        x_true.append(xs[i])
        epss.append(epsilon)
        testss.append(i)
for i in range(len(x_true)):
    errors.append(v_norm_2(x_my[i] - x_true[i].T()))

In [34]:
d = {
    'тест': testss,
    'Х настоящий': [x.T().array for x in x_true],
    'эпсилон': epss,
    'Х посчитанный': [x.T().array for x in x_my],
    'погрешность': errors,
    'итерации': k
}

In [35]:
pd.DataFrame(data=d)

,тест,Х настоящий,эпсилон,Х посчитанный,погрешность,итерации
0,0,"[[1.0], [2.0], [3.0]]",0.0100,"[[1.0004316117620913, 2.0008500909270106, 2.99...",0.001270,187
1,0,"[[1.0], [2.0], [3.0]]",0.0010,"[[0.9997925745720113, 2.0005859940219812, 2.99...",0.000704,268
2,0,"[[1.0], [2.0], [3.0]]",0.0001,"[[0.9999628556516078, 2.000082241288001, 2.999...",0.000100,908
3,1,"[[1.0], [1.0], [1.0]]",0.0100,"[[0.9964236955528054, 1.0007261420982685, 1.00...",0.003727,3
4,1,"[[1.0], [1.0], [1.0]]",0.0010,"[[0.9996930073966124, 1.0000903809565445, 1.00...",0.000324,5
5,1,"[[1.0], [1.0], [1.0]]",0.0001,"[[0.9999094817189167, 1.000028470906158, 1.000...",0.000096,6
6,2,"[[1.353], [1.308], [1.274]]",0.0100,"[[1.344086494155397, 1.304473257448219, 1.2715...",0.009887,7
7,2,"[[1.353], [1.308], [1.274]]",0.0010,"[[1.3519316145014044, 1.308173657689598, 1.273...",0.001082,11
8,2,"[[1.353], [1.308], [1.274]]",0.0001,"[[1.3524318060505707, 1.3084091135170102, 1.27...",0.000716,14
9,3,"[[1.3], [1.119], [1.082]]",0.0100,"[[1.294736618460355, 1.1141563952122977, 1.076...",0.008810,21


## Метод Зейделя

In [36]:
x_my = []
k = []
x_true = []
errors = []
epss = []
testss = []
for i in tests:
    for epsilon in epsilons:
        x_my.append(seidel(A[i], b[i], epsilon)[0])
        k.append(seidel(A[i], b[i], epsilon)[1])
        x_true.append(xs[i])
        epss.append(epsilon)
        testss.append(i)
for i in range(len(x_true)):
    errors.append(v_norm_2(x_my[i] - x_true[i].T()))

In [37]:
d = {
    'тест': testss,
    'Х настоящий': [x.T().array for x in x_true],
    'эпсилон': epss,
    'Х посчитанный': [x.T().array for x in x_my],
    'погрешность': errors,
    'итерации': k
}

In [38]:
pd.DataFrame(data=d)

,тест,Х настоящий,эпсилон,Х посчитанный,погрешность,итерации
0,0,"[[1.0], [2.0], [3.0]]",0.0100,"[[1.0105467865162971, 1.9775619939434304, 3.01...",2.726494e-02,209
1,0,"[[1.0], [2.0], [3.0]]",0.0010,"[[1.0010676910425307, 1.9977285158809401, 3.00...",2.760133e-03,333
2,0,"[[1.0], [2.0], [3.0]]",0.0001,"[[1.0001061083287208, 1.9997742573703503, 3.00...",2.743051e-04,458
3,1,"[[1.0], [1.0], [1.0]]",0.0100,"[[1.0003872884575573, 0.9998727602113282, 0.99...",4.079416e-04,2
4,1,"[[1.0], [1.0], [1.0]]",0.0010,"[[1.000010964366861, 1.0000002888409107, 0.999...",1.098813e-05,3
5,1,"[[1.0], [1.0], [1.0]]",0.0001,"[[1.0000000287009605, 1.0000000422168291, 0.99...",5.121922e-08,4
6,2,"[[1.353], [1.308], [1.274]]",0.0100,"[[1.3524861667201384, 1.3084388017655775, 1.27...",6.972627e-04,3
7,2,"[[1.353], [1.308], [1.274]]",0.0010,"[[1.3524861667201384, 1.3084388017655775, 1.27...",6.972627e-04,3
8,2,"[[1.353], [1.308], [1.274]]",0.0001,"[[1.352508527595025, 1.3084453723043181, 1.274...",6.856338e-04,4
9,3,"[[1.3], [1.119], [1.082]]",0.0100,"[[1.2997258375551564, 1.1188131410666418, 1.08...",3.831141e-04,15


## Метод Гаусса

In [39]:
x_my = []
x_true = []
errors = []
testss = []
for i in tests:
    x_my.append(gauss(A[i], b[i]))
    x_true.append(xs[i])
    testss.append(i)
for i in range(len(x_true)):
    errors.append(v_norm_2(x_my[i] - x_true[i].T()))

In [40]:
d = {
    'тест': testss,
    'Х настоящий': [x.T().array for x in x_true],
    'Х посчитанный': [x.T().array for x in x_my],
    'погрешность': errors
}

In [41]:
pd.DataFrame(data=d)

,тест,Х настоящий,Х посчитанный,погрешность
0,0,"[[1.0], [2.0], [3.0]]","[[1.0, 2.0, 3.0]]",0.000000
1,1,"[[1.0], [1.0], [1.0]]","[[1.0, 1.0, 1.0]]",0.000000
2,2,"[[1.353], [1.308], [1.274]]","[[1.3525091799265607, 1.3084455324357405, 1.27...",0.000685
3,3,"[[1.3], [1.119], [1.082]]","[[1.2997485067588808, 1.1188305564287961, 1.08...",0.000367
4,4,"[[-4.0], [-0.667], [6.25]]","[[-3.9999999999999982, -0.6666666666666665, 6....",0.000333


## Метод Хаусхолдера

In [42]:
x_my = []
x_true = []
errors = []
testss = []
for i in tests:
    x_my.append(householder(A[i], b[i]))
    x_true.append(xs[i])
    testss.append(i)
for i in range(len(x_true)):
    errors.append(v_norm_2(x_my[i] - x_true[i].T()))

In [43]:
d = {
    'тест': testss,
    'Х настоящий': [x.T().array for x in x_true],
    'Х посчитанный': [x.T().array for x in x_my],
    'погрешность': errors
}

In [44]:
pd.DataFrame(data=d)

,тест,Х настоящий,Х посчитанный,погрешность
0,0,"[[1.0], [2.0], [3.0]]","[[0.9999999999999982, 2.0000000000000036, 2.99...",4.550560e-15
1,1,"[[1.0], [1.0], [1.0]]","[[1.0000000000000002, 0.9999999999999993, 0.99...",1.132210e-15
2,2,"[[1.353], [1.308], [1.274]]","[[1.3525091799265605, 1.3084455324357405, 1.27...",6.852826e-04
3,3,"[[1.3], [1.119], [1.082]]","[[1.2997485067588808, 1.118830556428796, 1.082...",3.670807e-04
4,4,"[[-4.0], [-0.667], [6.25]]","[[-3.9999999999998987, -0.6666666666666674, 6....",3.333333e-04


In [148]:
# Test 5
n = 3
def bad_matr(n):
    E = Array(np.eye(n))
    A = Array(np.zeros((n,n)))
    B = Array(np.ones((n,n)) * eps * N)
    for i in range(n):
        for j in range(n):
            if i < j:
                A.array[i][j] = -1
                B.array[i][j] = B.array[i][j] * (-1)
    print(B.array)
    return E + A + B

In [ ]:
x_my = []
k = []
x_true = []
errors = []
epss = []
testss = []
for i in tests:
    for epsilon in epsilons:
        x_my.append(simple_iteration(A[i], b[i], epsilon)[0])
        k.append(simple_iteration(A[i], b[i], epsilon)[1])
        x_true.append(xs[i])
        epss.append(epsilon)
        testss.append(i)
for i in range(len(x_true)):
    errors.append(v_norm_2(x_my[i] - x_true[i].T()))